# NLP with pretrained models - spaCy and StanfordNLP

In [19]:
import spacy

#en = spacy.load("en")
en = spacy.load("en_core_web_lg") 
#en = spacy.load("en_core_web_sm")

In [95]:
path = "/Users/jlheller/Downloads/Discovering Sub-Networks in SBML Models/main.tex"
with open(path, 'r') as fd:
    texts = fd.readlines()
texts = [l.strip() for l in texts]
texts = [l for l in texts if (len(l) > 0) and (l[0] != "%") and (l[0] != "\\")]
texts[300:301]

['Unparenthesized numbers are the integer part $log_{10}$ of the median number of mappings of 1,000 simulations; parenthesized numbers are the 10\\% and 90\\% percentiles of 1,000 simulations.}']

In [94]:
# Eliminate control tokens
texts = '\nl'.join(texts).split()
texts = [t for t in texts if (not '#' in t) and (not '\\' in t)]
texts

['llcolorlinks,',
 'lllinkcolor={red!50!black},',
 'llcitecolor={blue!50!black},',
 'llurlcolor={blue!80!black}',
 'll}',
 'll}',
 'll}',
 'll}',
 'll}',
 'll}',
 'llinterconnections',
 'lbetween',
 'lelements',
 'lin',
 'lbiological',
 'lsystems',
 'l(e.g.,',
 'lstructural',
 'lanalysis',
 'lof',
 'lproteins',
 'lto',
 'linfer',
 'ltheir',
 'lfunction).',
 'llA',
 'lstrong',
 'lappeal',
 'lof',
 'lstructural',
 'lanalysis',
 'lis',
 'lthat',
 'llstructural',
 'linformation',
 'lis',
 'lgenerally',
 'lmore',
 'laccurate',
 'land',
 'lis',
 'lmore',
 'leasily',
 'lobtained',
 'lthan',
 'ldynamical',
 'ldata',
 'lsuch',
 'las',
 'lreaction',
 'lfluxes.',
 'llOur',
 'lfocus',
 'lis',
 'lon',
 'lsubnet',
 'ldiscovery}',
 'lin',
 'lchemical',
 'lreaction',
 'lnetworks',
 'l(CRNs)--discovering',
 'la',
 'lsubset',
 'lof',
 'la',
 'ltarget',
 'lCRN',
 'lthat',
 'lis',
 'llstructurally',
 'lidentical',
 'lto',
 'la',
 'lreference',
 'lCRN.',
 'llStructural',
 'lanalysis',
 'ltechniques',
 'lsu

In [80]:
doc_en = en('\nl'.join(texts))

First spaCy splits your document into sentences, and the sentences in tokens.

In [81]:
list(doc_en.sents)[0:5]

[Eq.~\ref{#1}}
 lFig.~\ref{#1}}
 lTab.~\ref{#1}}
 lcolorlinks,
 llinkcolor={red!50!black},
 lcitecolor={blue!50!black},
 lurlcolor={blue!80!black}
 l}
 l}
 l}
 l}
 l}
 l}
 linterconnections between elements in biological systems (e.g., structural analysis of proteins to infer their function).,
 lA strong appeal of structural analysis is that
 lstructural information is generally more accurate and is more easily obtained than dynamical data such as reaction fluxes.,
 lOur focus is on {\bf subnet discovery} in chemical reaction networks (CRNs)--discovering a subset of a target CRN that is
 lstructurally identical to a reference CRN.,
 lStructural analysis techniques such as
 lmotif finding and graph mining look for small, arbitrary, and commonly occurring substructures (e.g., 3 gene feedforward loops).,
 lIn contrast,
 lsubnet discovery looks for larger, specific, and infrequently occurring substructures (e.g., 10 reaction mitogen-activated protein kinase (MAPK) pathway).]

In [34]:
from IPython.display import HTML, display
import tabulate

tokens = [[token] for token in doc_en]
#display(HTML(tabulate.tabulate(tokens, tablefmt='html')))

In addition, spaCy also identifies a number of linguistic features for every token. The most basic of these are the lemma, and two types of parts-of-speech tags: the `pos_` attribute contains the [Universal POS tags](https://universaldependencies.org/u/pos/) from the [Universal Dependencies](https://universaldependencies.org/), while the `tag_` attribute contains more fine-grained, language-specific part-of-speech tags.

In [86]:
features = [[t.orth_, t.lemma_, t.pos_, t.tag_] for t  in doc_en]
features = [[t.orth_, t.lemma_] for t  in doc_en]
display(HTML(tabulate.tabulate(features, tablefmt='html')))

Eq.~\ref{#1,Eq.~\ref{#1
},}
},}
,
lFig.~\ref{#1,lfig.~\ref{#1
},}
},}
,
lTab.~\ref{#1,lTab.~\ref{#1
},}
},}


Next, spaCy also offers pre-trained models for named entity recognition. Their results can be found on the `ent_iob_` and `ent_type` attributes. The `ent_type` attribute tells us what type of entity the token refers to. In the English models, these entity types follow the [OntoNotes standard](https://catalog.ldc.upenn.edu/docs/LDC2013T19/OntoNotes-Release-5.0.pdf). In our example, we see that `Donald John Trump` refers to a person, `June 14, 1946` to a date, `45th` to an ordinal number, and `the United States` to a geo-political entity (GPE). 

The letters on the `ent_iob_` attribute give the position of the token in the entity. `O` means the token is outside of an entity, `B` means the token is at the beginning of an entity, and `I` means it is inside an entity (at any position except for the beginning). In this way, we can tell apart several entities of the same type that immediately follow each other. Together these letters form the so-called `BIO` tagging scheme. There are other tagging schemes, such as `BILUO`, which also has letters for the last position and single (unique) tokens in an entity, but the BIO scheme gives you all the information you need.  

In [9]:
entities = [(t.orth_, t.ent_iob_, t.ent_type_) for t in doc_en]
display(HTML(tabulate.tabulate(entities, tablefmt='html')))

Donald,B,PERSON
John,I,PERSON
Trump,I,PERSON
(,O,
born,O,
June,B,DATE
14,I,DATE
",",I,DATE
1946,I,DATE
),O,
is,O,


You can also access the entities directly on the `ents` attribute of the document: 

In [10]:
print([(ent.text, ent.label_) for ent in doc_en.ents])

[('Donald John Trump', 'PERSON'), ('June 14, 1946', 'DATE'), ('45th', 'ORDINAL'), ('the United States', 'GPE')]


Finally, spaCy also contains a dependency parser, which analyzes the grammatical relations between the tokens. 

In [11]:
syntax = [[token.text, token.dep_, token.head.text ] for token in doc_en]
display(HTML(tabulate.tabulate(syntax, tablefmt='html')))

Donald,compound,Trump
John,compound,Trump
Trump,nsubj,is
(,punct,Trump
born,acl,Trump
June,npadvmod,born
14,nummod,June
",",punct,June
1946,nummod,June
),punct,Trump
is,ROOT,is


## Multingual NLP

SpaCy doesn't only have models for English, but also for many other languages. Here's an example of a Dutch sentence, which means "Charles Michel is the prime minister of Belgium".

In [12]:
#nl = spacy.load("nl")
#text_nl = "Charles Michel is de eerste minister van België."
#doc_nl = nl(text_nl)

The tokens in the Dutch document have the same attributes as those in the English one. Take care, however, because the functionality of the models can differ across languages. Here are three main differences between the English and the Dutch model: 

- The Dutch model does not offer lemmatization: the lemma_ attribute is identical to the orth_ attribute.
- The Dutch model has a very different fine-grained part-of-speech tags on the tag_ attribute.
- The Dutch model has different entity types (PER, LOC and ORG) than the English one. 

This is a result of the training corpora that were used to build the models, whose annotation guidelines may be very different.

In [13]:
#info = [(t.orth_, t.lemma_, t.pos_, t.tag_, t.ent_iob_, t.ent_type_) for t in doc_nl]
#display(HTML(tabulate.tabulate(info, tablefmt='html')))

## StanfordNLP

Another library whose functionality overlaps with that of spaCy is StanfordNLP. [StanfordNLP](https://stanfordnlp.github.io/stanfordnlp/), not to be confused with Stanford's Java [CoreNLP](https://stanfordnlp.github.io/CoreNLP/) library, is a [Python library](https://github.com/stanfordnlp/stanfordnlp) built on top of PyTorch that offers a fully neural pipeline with tokenization (including multi-word units), lemmatization, part-of-speech tagging (including morphological features) and dependency parsing. These components were built and trained for the [CoNLL-2018 shared task](https://nlp.stanford.edu/pubs/qi2018universal.pdf). There are no named entities, but the quality of the dependency parsing is state of the art. On top of that, it also offers a Python interface to CoreNLP. 

Its API is very similar to that of spaCy:

In [14]:
#import stanfordnlp

#stanfordnlp.download('nl')
#nl_stanford = stanfordnlp.Pipeline(lang="nl")

In [15]:
#doc_nl_stanford = nl_stanford(text_nl)

The `text` and `lemma` properties speak for themselves. The `upos` attribute contains the universal dependencies we also find on spaCy's `pos_` attribute; the `xpos` attribute corresponds to spaCy's `tag_` attribute and contains the fine-grained tags with morphological properties. The `governor` attribute contains the (1-based) index of the head of each token; `dependency_relation` contains the grammatical relation between the two. 

In [16]:
if False:
    stanford_info = []
    for sentence in doc_nl_stanford.sentences:
        for token in sentence.tokens:
            for word in token.words:
                stanford_info.append((len(stanford_info)+1, word.text, word.lemma, word.upos, word.xpos, word.dependency_relation, word.governor))

In [17]:
#display(HTML(tabulate.tabulate(stanford_info, tablefmt='html')))

## Combining spaCy and StanfordNLP

**Had difficulties with StanfordNLP**

If you can't choose between spaCy and StanfordNLP, you can also combine the two. Yes, sometimes you can have the best of both worlds. Thanks to the `spacy_stanfordnlp` wrapper, we can plug a Stanford NLP model into a spaCy pipeline, and get its annotations in a spaCy document. Suddenly we have Dutch lemmatization and state-of-the-art part-of-speech tagging and dependency parsing.

In [18]:
from spacy_stanfordnlp import StanfordNLPLanguage
import stanfordnlp

nl_stanford = stanfordnlp.Pipeline()
nl_combined = StanfordNLPLanguage(nl_stanford)

doc_nl_combined = nl_combined(text_nl)

info = [(t.orth_, t.lemma_, t.pos_, t.tag_) for t in doc_nl_combined]
display(HTML(tabulate.tabulate(info, tablefmt='html')))

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/Users/jlheller/stanfordnlp_resources/en_ewt_models/en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Cannot load model from /Users/jlheller/stanfordnlp_resources/en_ewt_models/en_ewt_tokenizer.pt


AttributeError: 'tuple' object has no attribute 'tb_frame'

Additionally, we can mix and match the strengths of the two libraries, for example by extending our pipeline with spaCy's Named Entity Recognition.

In [ ]:
nl_combined = StanfordNLPLanguage(nl_stanford)
nl_ner = nl.get_pipe("ner")
nl_combined.add_pipe(nl_ner)
nl_combined.vocab.strings.add("PER")

doc_nl_combined = nl_combined(text_nl)

info = [(t.orth_, t.lemma_, t.pos_, t.tag_, t.ent_iob_, t.ent_type_) for t in doc_nl_combined]
display(HTML(tabulate.tabulate(info, tablefmt='html')))